In [175]:
import pandas as pd
import theano
import lasagne
import numpy as np
from theano.tensor import *
import time

train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

labelnumber=5   # 0 to 4

In [176]:
# convert from dataframe to numpy array
y_train = train.y.as_matrix()

X_train = train.iloc[:,1:101].as_matrix()
X_test= test.as_matrix()
X = np.concatenate((X_train, X_test), 0)

print(X.shape)

# drop feature with too small standard deviation
drop_feature = np.where(X.std(axis=0)<0.005)
X = np.delete(X, drop_feature, axis=1)

dimension = X.shape[1]
print ("feature selected new dimension is", int(dimension))

from sklearn.preprocessing import scale
X = scale(X, axis=0)
X_train = X [0:X_train.shape[0],:]
X_test  = X [X_train.shape[0]:X.shape[0],:]

# divide into train set and eval set
num_sample = y_train.shape[0]

# turn y_train into binary matrix
# here Y is capital because a matrix
Y_train = np.zeros((y_train.shape[0],5))
Y_train[np.arange(0, num_sample), y_train] = 1

# take 1/8 for evaluation
num_eval = np.rint(num_sample/8)  
num_train = num_sample-num_eval


X_val = X_train[num_train:num_sample]
X_train = X_train[0:num_train]
Y_val = Y_train[num_train:num_sample].astype(int) 
Y_train = Y_train[0:num_train].astype(int) 

print('train set size', X_train.shape, Y_train.shape)
print('test set size', X_val.shape, Y_val.shape)

(53461L, 100L)
('feature selected new dimension is', 79)
('train set size', (39659L, 79L), (39659L, 5L))
('test set size', (5665L, 79L), (5665L, 5L))


D:\Program Files\Miniconda\lib\site-packages\ipykernel\__main__.py:35: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
D:\Program Files\Miniconda\lib\site-packages\ipykernel\__main__.py:36: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
D:\Program Files\Miniconda\lib\site-packages\ipykernel\__main__.py:37: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
D:\Program Files\Miniconda\lib\site-packages\ipykernel\__main__.py:38: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [177]:
def build_mlp(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, dimension),
                                     input_var=input_var)
    
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)
    
    l_hid1 = lasagne.layers.DenseLayer(
        l_in_drop, num_units=200,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    l_hid2 = lasagne.layers.DenseLayer(
        l_hid1_drop, num_units=200,
        nonlinearity=lasagne.nonlinearities.rectify)

    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)
    
    l_out = lasagne.layers.DenseLayer(
        l_hid2_drop, num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_out

In [178]:
# Prepare Theano variables for inputs and targets
input_var = dmatrix('input_var')
target_var = imatrix('target_var')
# Create neural network model

network = build_mlp(input_var)
prediction = lasagne.layers.get_output(network)
# define loss function
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

# update weights
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

test_prediction = lasagne.layers.get_output(network, deterministic=True)


test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [179]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [180]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
num_epochs = 10
batchsize = 500
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, Y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, Y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))